#### Regression
以'508056.SH_close'的ln return为例

In [8]:
import pandas as pd
fac_w_hf_adj_2 = pd.read_csv('ga_subset_factor_data.csv',index_col=[0])
close_df = pd.read_csv('D:\cicc_reits_pred\close_df.csv',index_col=[0])

In [10]:
import numpy as np
import math
def get_ln_ret(a, b):
    if np.isnan(b):
        return b
    if math.isclose(b, 0, abs_tol=1e-15) == True:
        return np.nan
    return np.log(a/b)
data = fac_w_hf_adj_2
close_name_temp = '508056.SH_close'
close_df_temp = close_df.copy(deep=True)
close_df_temp[close_name_temp+'_shift'] = close_df_temp[close_name_temp].shift(1)
close_df_temp[close_name_temp+'_chg'] = close_df_temp.apply(lambda x: get_ln_ret(x[close_name_temp], x[close_name_temp+'_shift']), axis=1)

data = pd.merge(data,close_df_temp[['time',close_name_temp+'_chg']],on='time',how='inner')

data_new = data.dropna(subset=[close_name_temp+'_chg'])
data_new.index = range(len(data_new))

In [13]:
import statsmodels.api as sm
reg_result = pd.DataFrame(columns=['R_sq','t_value','p_value'],index=list(data_new.columns[1:-1]))
for col in data_new.columns[1:-1]:
    temp = data_new[[col,close_name_temp+'_chg']].dropna(how='any',axis=0)
    x = temp[[col]]
    x = sm.add_constant(x)
    y = temp[[close_name_temp+'_chg']]
    model = sm.OLS(y, x.astype(float)).fit()
    reg_result.loc[col,'R_sq'] = model.rsquared
    reg_result.loc[col,'t_value'] = model.tvalues[col]
    reg_result.loc[col,'p_value'] = model.pvalues[col]

In [14]:
multi_lin_reg_result = pd.DataFrame(columns=['R_sq','t_value','p_value'],index=list(data_new.columns[1:-1]))
x = data_new.iloc[:,1:-1].fillna(data_new.iloc[:,1:-1].mean())
x = sm.add_constant(x)
y = data_new[[close_name_temp+'_chg']]
model = sm.OLS(y, x.astype(float)).fit()
for col in data_new.columns[1:-1]:
    multi_lin_reg_result.loc[col,'R_sq'] = model.rsquared
    multi_lin_reg_result.loc[col,'t_value'] = model.tvalues[col]
    multi_lin_reg_result.loc[col,'p_value'] = model.pvalues[col]

In [15]:
multi_lin_reg_result = multi_lin_reg_result.sort_values(by=['p_value','R_sq'],ascending=True)
multi_lin_reg_result[multi_lin_reg_result['p_value']<0.05]

,R_sq,t_value,p_value
reits_ind_park_close,0.225618,-3.050053,0.002432
180201.SZ_ind_all_dacce_prev,0.225618,-2.524823,0.01194
508000.SH_rskew_prev,0.225618,-2.448353,0.014756
000688.SH_turnover,0.225618,-2.314124,0.021139
180801.SZ_large_trans_ret_prev,0.225618,-2.312433,0.021233
508018.SH_large_trans_ret_prev,0.225618,-2.20721,0.027836
508018.SH_reverse_prev,0.225618,2.074235,0.038662
180101.SZ_ind_park_dacce_prev,0.225618,-2.070535,0.039008


In [16]:
reg_result = reg_result.sort_values(by=['p_value','R_sq'],ascending=True)

In [17]:
reg_result[reg_result['p_value']<0.05]

,R_sq,t_value,p_value
180101.SZ_ind_park_dacce_prev,0.012525,-2.508189,0.012453
180201.SZ_reverse_prev,0.025599,2.442068,0.015368
508027.SH_ind_park_dacce_prev,0.011296,-2.380508,0.017665
000832.CSI_amt,0.010033,2.2421,0.025396
508000.SH_rskew_prev,0.021577,-2.232472,0.026565
508008.SH_reverse_prev,0.020734,2.192334,0.029371
508018.SH_ind_highway_diff_prev,0.015965,2.169123,0.030886
508088.SH_large_trans_ret_prev,0.023721,2.085483,0.038444
reits_ind_park_close,0.008532,-2.070173,0.038951
180202.SZ_large_trans_ret_prev,0.018182,2.05031,0.041483


In [18]:
reg_result.to_csv('D:\\REITS研究数据\\single_lreg_result_ga_subset.csv')
multi_lin_reg_result.to_csv('D:\\REITS研究数据\\multi_lreg_result_ga_subset.csv')